In [5]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [6]:
from eolearn.core import (
    AddFeatureTask,
    EONode,
    EOPatch,
    EOTask,
    EOWorkflow,
    FeatureType,
    LoadTask,
    OverwritePermission,
    SaveTask,
)

In [7]:
import datetime
import os
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import Polygon

In [8]:
from pathlib import Path
import sys

SCRIPT_DIR = Path('/home/alina.smolina/eolearn-pipeline/src')
sys.path.append(str(SCRIPT_DIR))
SCRIPT_DIR = Path('/home/alina.smolina/DL-pipeline/src')
sys.path.append(str(SCRIPT_DIR))

In [9]:
eopatches_dir = '/beegfs/home/alina.smolina/data/sakhalin/images/EOPatches/train_2560_K_N_v4/'
num_ids_train = 598


eopatches_dir_test = '/beegfs/home/alina.smolina/data/sakhalin/images/EOPatches/test_2560_Nevelsk_v4/'
num_ids_test = 44

## Preparing dataloader

In [10]:
eopatch = EOPatch.load(f'{eopatches_dir}eopatch_20', lazy_loading=True)

## Model

In [11]:
from datamodule_eolearn import GeoEOModule

import torch
torch.set_float32_matmul_precision('high')

In [12]:
import albumentations as A
import albumentations.pytorch as AP

transforms = A.Compose([
    A.Flip(p=0.3),
    A.ShiftScaleRotate(
        shift_limit=(-0.0625, 0.0625), 
        scale_limit=0, #no scale
        rotate_limit=(-90, 90), 
        p=0.5
    ),
    AP.ToTensorV2(transpose_mask=True),
    ],
)

test_transform = A.Compose([
    AP.ToTensorV2(transpose_mask=True),
    ],
)

In [51]:
target_name = 'polnota'


all_dates = GeoEOModule(
    transform = transforms,
    test_transform = test_transform,
    target_mask_name = target_name,
    train_eopatches_dir = eopatches_dir, 
    test_eopatches_dir = eopatches_dir_test, 
    predict_eopatches_dir = eopatches_dir_test, 
    train_eopatches_ids = [x for x in range(num_ids_train)], # [x for x in range(40)],   #<------- ONLY FOR DEBUG
    test_eopatches_ids = [x for x in range(num_ids_test)], 
    predict_eopatches_ids = [x for x in range(num_ids_test)], 
    train_date_range = ['2018-05-30', '2018-08-31'],
    test_date_range = ['2018-05-30', '2018-08-31'],
    predict_date_range = ['2018-05-30', '2018-08-31'],
    batch_size = 128,
    num_workers = 16,
    target_feature_name = FeatureType.DATA_TIMELESS,
)


##### TIP: для target_name = 'высота'
##### надо поставить target_feature_name = FeatureType.MASK_TIMELESS


In [54]:
import wandb
import pytorch_lightning as pl
from regression_module import RegModule
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import LearningRateMonitor


import os
os.environ['WANDB__SERVICE_WAIT'] = '300'
wandb_logger = WandbLogger(project=f'dl-pipeline-sakhalin-{target_name}', log_model=True)
print(f'🤖: Look at {wandb.run.url}')


lr_monitor_callback = LearningRateMonitor(logging_interval='step')


checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath=f'/beegfs/home/alina.smolina/DL-pipeline/weights/{target_name}-regression-sakhalin/{wandb.run.name}/',
    filename='{epoch}-{val/loss:.4f}-{val/mape:.4f}', 
    monitor='val/mape',
    mode='min',
    save_top_k=1
)

                   
trainer = pl.Trainer(
    max_epochs=1000, 
    benchmark=True, 
    check_val_every_n_epoch=10, 
    logger=wandb_logger, 
    callbacks=[
        checkpoint_callback,
        lr_monitor_callback,
              ],
)

/trinity/home/alina.smolina/.conda/envs/geotorch/lib/python3.10/site-packages/pytorch_lightning/loggers/wandb.py:398: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
/trinity/home/alina.smolina/.conda/envs/geotorch/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:168: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /trinity/home/alina.smolina/.conda/envs/geotorch/lib ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


🤖: Look at https://wandb.ai/nali/dl-pipeline-sakhalin-polnota/runs/4xd65hz9


In [57]:
model = RegModule(
    optimizer='Adam', 
    scheduler='StepLR',
    step_size=100,
    gamma=0.8,
    lr=6e-4,
    in_channels=10,
    ignore_index=0,
    min_channels=16,
    max_channels=512,
    num_down_blocks=5,
)

##### TIP: для target_name = 'высота' нужно ignore_index=255
trainer.fit(model, all_dates) 

/trinity/home/alina.smolina/.conda/envs/geotorch/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:168: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /trinity/home/alina.smolina/.conda/envs/geotorch/lib ...
  rank_zero_warn(


🤖: Setup data...
🤖: Fit stage.
🤖: Collecting all the time indices... Done.
🤖: Generating mapping: idx -> (patch_id, time_id)... Elapsed time: 0.28 min
Done.
🤖: Loading all the patches and time frames into the memory... 

/trinity/home/alina.smolina/.conda/envs/geotorch/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:617: UserWarning: Checkpoint directory /beegfs/home/alina.smolina/DL-pipeline/weights/polnota-regression-sakhalin/polished-sunset-2 exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name       | Type                        | Params
------------------------------------------------------------
0  | net        | UNet                        | 16.5 M
1  | loss_fn    | MSELoss                     | 0     
2  | train_mape | MeanAbsolutePercentageError | 0     
3  | train_mae  | MeanAbsoluteError           | 0     
4  | train_mse  | MeanSquaredError            | 0     
5  | train_r2   | R2Score                     | 0     
6  | val_mape   | MeanAbsolutePercentageError | 0     
7  | val_mae    | MeanAbsoluteError           | 0     
8  | val_mse    | MeanSquaredError       

Done.
Elapsed time: 0.19 min 🤖: Fit stage.
🤖: Fit frames >>> 45
🤖: Val frames >>> 11
🤖: Total >>>>>>>> 56


Sanity Checking: 0it [00:00, ?it/s]

...

🤖: #iterations in train dataloader: 1


/trinity/home/alina.smolina/.conda/envs/geotorch/lib/python3.10/site-packages/pytorch_lightning/loops/fit_loop.py:281: PossibleUserWarning: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x155484967640>
Traceback (most recent call last):
  File "/trinity/home/alina.smolina/.conda/envs/geotorch/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1466, in __del__
    self._shutdown_workers()
  File "/trinity/home/alina.smolina/.conda/envs/geotorch/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1430, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/trinity/home/alina.smolina/.conda/envs/geotorch/lib/python3.10/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
  File "/trinity/home/alina.smolina/.conda/envs/geotorch/lib/python3.10/multiprocessing/popen_fork.py", line 40, in wait
    if not wait([self.sentinel], timeout):
  File "/trinity/home/alina.smolina/.conda/envs/geotorch/lib/python3.10/multiprocessing/connection.py", line 931, in wait
    ready = selector.select(timeout)
  File "/trinity/home/al

RuntimeError: DataLoader worker (pid(s) 3215658, 3215722, 3215786, 3215850, 3215914, 3215978, 3216042, 3216106, 3216171, 3216235, 3216299, 3216363, 3216427, 3216491, 3216555, 3216619) exited unexpectedly

In [58]:
trainer.test(model, all_dates)

/trinity/home/alina.smolina/.conda/envs/geotorch/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:168: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /trinity/home/alina.smolina/.conda/envs/geotorch/lib ...
  rank_zero_warn(


🤖: Setup data...
🤖: Collecting all the time indices... Done.
🤖: Generating mapping: idx -> (patch_id, time_id)... Elapsed time: 0.48 min
Done.
🤖: Loading all the patches and time frames into the memory... 

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Done.
Elapsed time: 0.81 min 🤖: Test stage.
🤖: Test frames >>> 165
🤖: #iterations in test dataloader: 2


/trinity/home/alina.smolina/.conda/envs/geotorch/lib/python3.10/site-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 10, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test/mae            0.5089307427406311
        test/mape           0.9198791980743408
        test/mse            0.2718241810798645
         test/r2            -20.069772720336914
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test/mape': 0.9198791980743408,
  'test/mae': 0.5089307427406311,
  'test/mse': 0.2718241810798645,
  'test/r2': -20.069772720336914}]

In [55]:
%%time
uniq_vals = []
# # for item in eo_ds:
all_dates.setup(stage='fit')
for item in all_dates.train_dataloader():
    print(len(item))
    print(item[0].shape)
    print(item[1].shape)
    uniq_vals.append(torch.unique(item[-1]).numpy())
    break

🤖: Setup data...
🤖: Fit stage.
🤖: Collecting all time indices... Done.
🤖: Generating mapping: idx -> (patch_id, time_id)... Done.
🤖: Fit stage.
🤖: Fit frames >>> 1485
🤖: Val frames >>> 371
🤖: Total >>>>>>>> 1856
🤖: #iterations in train dataloader: 62
3
torch.Size([24, 10, 256, 256])
torch.Size([24, 256, 256])
CPU times: user 22.7 s, sys: 1.93 s, total: 24.7 s
Wall time: 4min 34s


[rank: 0] Received SIGTERM: 15
[rank: 0] Received SIGTERM: 15
[rank: 0] Received SIGTERM: 15
[rank: 0] Received SIGTERM: 15
[rank: 0] Received SIGTERM: 15
[rank: 0] Received SIGTERM: 15


In [53]:
%%time
uniq_vals = []
# # for item in eo_ds:
all_dates.setup(stage='fit')
for item in all_dates.train_dataloader():
    print(len(item))
    print(item[0].shape)
    print(item[1].shape)
    uniq_vals.append(torch.unique(item[-1]).numpy())
    break

🤖: Setup data...
🤖: Fit stage.
🤖: Collecting all time indices... Done.
🤖: Generating mapping: idx -> (patch_id, time_id)... Done.
🤖: Fit stage.
🤖: Fit frames >>> 1485
🤖: Val frames >>> 371
🤖: Total >>>>>>>> 1856
🤖: #iterations in train dataloader: 62
3
torch.Size([24, 10, 256, 256])
torch.Size([24, 256, 256])
CPU times: user 22.7 s, sys: 1.03 s, total: 23.8 s
Wall time: 37.3 s


In [ ]:
uniq_vals

In [20]:
from datamodule_eolearn import EOLearnDataset

eo_ds = EOLearnDataset(
    transform = transforms,
    eopatches_dir = eopatches_dir,
    eopatches_ids = [x for x in range(69, 106)],
    date_range = ['2018-01-01', '2018-12-31'],
    target_mask_name = 'PREVAIL_GROUP_2',
    demand_target = True,
)

🤖: Collecting all time indices...
🤖: Done.


In [25]:
for item in eo_ds:
    print(len(item))
    print(item[0].shape)
    print(item[1].shape)
    break

3
torch.Size([10, 256, 256])
torch.Size([256, 256])
